# Reformatter and resampler of APPA's data
mainly by *Samuele Facenda*

copy-paste contributed by *Magoni Parker*


--- 

convert the table with a column for every channel to a table(with a table with the chenges of sensing materials) with a feature that is the sensing material, age of the sensor, sin and cos of date and a lot of other things


In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import locale
from tqdm import tqdm
import numpy as np
import math
from numpy import NaN

In [4]:
def elaborate_file(file_name, sensing_material_table_file):
    sensing_table = pd.read_csv(sensing_material_table_file)
    raw_table = pd.read_csv(file_name)
    # todo


### read the files ###

In [5]:
sensing_table = pd.read_csv("../sensing_material_appa1.csv")
raw_table = pd.read_csv("../appa1.csv")

the name of the column in the transform file is a string, i must have a list with this strings and a list with the convertion in timestamp

In [6]:
# get the timestamp of the sensing material change date
dates_str = sensing_table.columns.to_list()
dates_str.remove("nm")
dates = [pd.to_datetime(x) for x in dates_str]

#### refactor:
convert in timestamp and aggregate every hour with the mean

In [7]:
# change raw_table ts column to timestamp
raw_table.ts = pd.to_datetime(raw_table.ts.str.slice(start = 0, stop = 24), format = "%a %b %d %Y %H:%M:%S")
NUMBER_OF_SENSORS = 8

In [8]:
# set and reset index to computate the mea
raw_table = raw_table.set_index("ts")
raw_table = raw_table.resample("H").mean()
raw_table = raw_table.reset_index(level=0)

### utility functions:
get_sensing_mat:
- index_date is the number of column in the tranform file where to search
- channel is the channel to get

find_change_date:
- the index is the column of the file where to search in the channel
- channel is the channel

a little recursive function, if there is a * in the name recall himself with a lower index, if the lower index is a different molecule, check in the other channels

In [9]:
def get_sensing_mat(index_date, channel):
    return sensing_table[dates_str[index_date]][sensing_table.nm == channel].values[0]

def find_change_date(index_date_reach, channel):
    actual_sens_mat = get_sensing_mat(index_date_reach, channel)
    if  not '*' in actual_sens_mat:
        return dates[index_date_reach]
    else:
        # find the last change
        if get_sensing_mat(index_date_reach -1,channel) in actual_sens_mat:
            # do the same thing on the previus cell
            return find_change_date(index_date_reach - 1, channel)
        else:
            # the sensor changed channel, find the channel and do the same thing
            channel_before = 1
            while channel_before <= NUMBER_OF_SENSORS:
                if get_sensing_mat(index_date_reach -1,channel_before) in actual_sens_mat:
                    break
                channel_before += 1
            else:
                raise Exception(f"error in find previus position of sensor\n actual index {index_date_reach} type {actual_sens_mat}")
            return find_change_date(index_date_reach - 1, channel_before)


define the columns

create a new dataframe, the structure is too different, I can't keep the old one

keep track during the iteration of the material in every channel and the timestamp of his first installation

in the iteration(on every row, called record) check if the date is before the first certified installation(if so skip to the next row), check if a sensing material change day is reached(and then computate and update the traks array ex. lasta_change_data...), so for every channel create a row with all the columns and fill it. 

In the end of every iteration concatenate the temporary dataset to the final one

In [ ]:

columns = ['ts','sensing_,material','signal_res','heater_res','heater_V','T','RH','P','time_cos','time_sin','date_cos','date_sin','Y','age']
new_data = pd.DataFrame(columns=columns)

last_change_date = [dates[0] for _ in range(NUMBER_OF_SENSORS)]
index_date_reached = 0
actual_sensing_material = [get_sensing_mat(0,i) for i in range(1,NUMBER_OF_SENSORS + 1)]

for index, record in tqdm(raw_table.iterrows(), total = raw_table.shape[0]):
    if record.ts.date() < dates[0].date():
        # skip all data before sensor changing(and sensing mat knowing)
        continue

    if index_date_reached != len(dates) and record.ts.date() == dates[index_date_reached+1].date():
        # reached sensing material change day
        index_date_reached += 1
        for channel in range(NUMBER_OF_SENSORS):
            last_change_date[channel] = find_change_date(index_date_reached, channel+1)

        # update the sensing material array
        actual_sensing_material = [get_sensing_mat(index_date_reached,i) for i in range(1,NUMBER_OF_SENSORS + 1)]

    append_ds = pd.DataFrame(columns=columns)
    for channel in range(1,NUMBER_OF_SENSORS + 1):
        if record["Rs"+str(channel)] == np.NaN:
            continue
        # for every channel add a row
        append_ds.loc[len(append_ds.index)] = [
            record['ts'],
            actual_sensing_material[channel-1].replace("*",""),
            record['Rs'+str(channel)],
            record['Rh'+str(channel)],
            record['Vh'+str(channel)],
            record['T'],
            record['RH'],
            record['P'],
            math.cos(record['ts'].hour) * 2 * (math.pi / 24),
            math.sin(record['ts'].hour) * 2 * (math.pi / 24),
            math.cos(record['ts'].month) * 2 * (math.pi / 12),
            math.sin(record['ts'].month) * 2 * (math.pi / 12),
            record['ts'].year,
            record['ts'] - last_change_date[channel-1]
        ]
    new_data = new_data.append(append_ds)


Last clear and file/print

In [ ]:
new_data = new_data.dropna().reset_index(drop=True)

In [11]:
print(new_data)
new_data.to_csv("../appa1_new.csv", index = False)

           ts sensing_,material       signal_res  heater_res  heater_V      T  \
0  2021-01-13              ZnOR              NaN         NaN       NaN    NaN   
1  2021-01-13              ZnOR              NaN         NaN       NaN    NaN   
2  2021-01-13            LaFeO3              NaN         NaN       NaN    NaN   
3  2021-01-13            LaFeO3              NaN         NaN       NaN    NaN   
4  2021-01-13               WO3              NaN         NaN       NaN    NaN   
..        ...               ...              ...         ...       ...    ...   
3  2022-04-19              WO3*  48403051.966667      127.42  2.400167  45.66   
4  2022-04-19             ZnOR*      355588748.8  121.931667  3.980667  45.66   
5  2022-04-19             ZnOR*      365193468.8  120.658333  3.985167  45.66   
6  2022-04-19           SmFeO3*       39482509.8   96.103333  3.579833  45.66   
7  2022-04-19              STN*      198154600.8  111.623333     4.263  45.66   

          RH      P  time_c